### Bias Field Correction

**Learning outcomes:**
- Learn how to apply Bias Field Correction to an image using SITK.
- Inspect Bias Field visually.

In [1]:
%matplotlib inline

import os
from help_functions import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.4.2
SimpleITK version = 2.3.1


In [2]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
print(f'project folder = {BASE_DIR}')

project folder = c:\Users\Windows\Desktop\GBM_Segmentation


In [3]:
raw_img_path = os.path.join(BASE_DIR, 'Brain_Extraction', 'sub-01', 'anat', 'sub-01_T1w.nii')
print(f'raw_img_path = {raw_img_path}')

raw_img_path = c:\Users\Windows\Desktop\GBM_Segmentation\Brain_Extraction\sub-01\anat\sub-01_T1w.nii


#### Load image

In [4]:
raw_img_sitk = sitk.ReadImage(raw_img_path, sitk.sitkFloat32)
raw_img_sitk = sitk.DICOMOrient(raw_img_sitk,'RPS')

raw_img_sitk_arr = sitk.GetArrayFromImage(raw_img_sitk)
print(f'shape = {raw_img_sitk_arr.shape} -> (Z, X, Y)')
explore_3D_array(raw_img_sitk_arr, cmap='nipy_spectral')

shape = (320, 300, 208) -> (Z, X, Y)


interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Create head mask

In [5]:
transformed = sitk.RescaleIntensity(raw_img_sitk, 0, 255)

#transformed = sitk.TriangleThreshold(transformed, 0, 1)
transformed = sitk.LiThreshold(transformed,0,1)

head_mask = transformed

explore_3D_array_comparison(
    arr_before=sitk.GetArrayFromImage(raw_img_sitk),
    arr_after=sitk.GetArrayFromImage(head_mask)
)

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Bias Correction

In [6]:
shrinkFactor = 4
inputImage = raw_img_sitk

inputImage = sitk.Shrink( raw_img_sitk, [ shrinkFactor ] * inputImage.GetDimension() )
maskImage = sitk.Shrink( head_mask, [ shrinkFactor ] * inputImage.GetDimension() )

bias_corrector = sitk.N4BiasFieldCorrectionImageFilter()

corrected = bias_corrector.Execute(inputImage, maskImage)

#### Get image corrected

In [7]:
log_bias_field = bias_corrector.GetLogBiasFieldAsImage(raw_img_sitk)
corrected_image_full_resolution = raw_img_sitk / sitk.Exp( log_bias_field )

explore_3D_array_comparison(
    sitk.GetArrayFromImage(raw_img_sitk),
    sitk.GetArrayFromImage(corrected_image_full_resolution), 
    cmap='nipy_spectral')


interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Inspect the bias field

In [8]:
# bias field
temp = sitk.Exp(log_bias_field)
temp = sitk.Mask(temp, head_mask)
explore_3D_array(sitk.GetArrayFromImage(temp), cmap='gray')

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Save the image

In [10]:
out_folder =  os.path.join(BASE_DIR, 'Brain_Extraction', 'preprocessed')
out_folder = os.path.join(out_folder, 'Preprocessed_image') # create folder with name of the raw file
os.makedirs(out_folder, exist_ok=True) # create folder if not exists

out_filename = add_suffix_to_filename('sub-01_T1w.nii', suffix='biasFieldCorrected')
out_path = os.path.join(out_folder, out_filename)

print(raw_img_path[len(BASE_DIR):])
print(out_path[len(BASE_DIR):])

\Brain_Extraction\sub-01\anat\sub-01_T1w.nii
\Brain_Extraction\preprocessed\Preprocessed_image\sub-01_T1w_biasFieldCorrected.nii


In [11]:
sitk.WriteImage(corrected_image_full_resolution, out_path)